<a href="https://colab.research.google.com/github/ebtrader/test/blob/master/price_channel_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import numba
from scipy.stats import linregress
from tqdm import tqdm
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
@numba.njit
def shift_nb(arr, num, fill_value=np.nan):
    if num >= 0:
        return np.concatenate((np.full(num, fill_value), arr[:-num]))
    else:
        return np.concatenate((arr[-num:], np.full(-num, fill_value)))

def get_ranked_extrema(h, l, max_neighbours=10):
    maxima = np.zeros(shape=(h.shape[0]), dtype=int)
    minima = np.zeros(shape=(h.shape[0]), dtype=int)

    for n in range(1, max_neighbours+1):
        maxima_filter = np.where(maxima == n-1)[0]
        maxima[maxima_filter] += ((h >= shift_nb(h, n)) & (h > shift_nb(h, -n)))[maxima_filter]

        minima_filter = np.where(minima == n-1)[0]
        minima[minima_filter] += ((l <= shift_nb(l, n)) & (l < shift_nb(l, -n)))[minima_filter]

    return maxima, minima

def get_best_channel(
        h, l, c, max_rank_possible=20, min_rank_for_detection=3,
        min_pivot_point_count=3, max_age_latest_pivot_point=20,
        min_r_squared=0.9, max_p_value=0.5, max_crossing_perc=0.05,
        min_pivot_points_inbetween=1):
    # Calculate the rank of each high and low
    all_maxima, all_minima = get_ranked_extrema(h, l, max_rank_possible)

    # Filter out minima whose rank is too low
    all_minima_x = np.where(all_minima >= min_rank_for_detection)[0]
    all_minima_y = l[all_minima_x]
    all_minima_weights = all_minima[all_minima_x]

    # Filter out maxima whose rank is too low
    all_maxima_x = np.where(all_maxima >= min_rank_for_detection)[0]
    all_maxima_y = h[all_maxima_x]
    all_maxima_weights = all_maxima[all_maxima_x]

    best_lower_trend_line = None
    best_lower_trend_stats = None
    best_upper_trend_line = None
    best_upper_trend_stats = None
    best_channel_score = -1

    # "Sort minima by their rank and after each iteration remove the minimum with the lowest rank"
    for minima_count in range(min_pivot_point_count, all_minima_x.shape[0]+1):
        minima_comb = np.argpartition(all_minima_weights, -minima_count)[-minima_count:]

        # Get combination of minima
        minima_x = all_minima_x[minima_comb]
        minima_y = all_minima_y[minima_comb]
        minima_weights = all_minima_weights[minima_comb]

        # Do linear regression for these minima
        min_slope, min_intercept, min_r, min_p, min_err = linregress(minima_x, minima_y)
        # The coefficient of determination needs to be high enough
        if min_r*min_r <= min_r_squared or min_p >= max_p_value:
            continue

        # Calculate the regression line
        min_line_x = np.arange(np.min(minima_x), c.shape[0])
        min_line_y = min_intercept + min_slope * min_line_x
        # Calculate the crossing price percentage
        min_crossing_price_perc = np.mean(c[min_line_x] < min_line_y)
        # This needs to be sufficiently small
        if min_crossing_price_perc >= max_crossing_perc:
            continue

        # "Sort maxima by their rank and after each iteration remove the maximum with the lowest rank"
        for maxima_count in range(min_pivot_point_count, all_maxima_x.shape[0]+1):
            maxima_comb = np.argpartition(all_maxima_weights, -maxima_count)[-maxima_count:]

            # Get combination of maxima
            maxima_x = all_maxima_x[maxima_comb]
            maxima_y = all_maxima_y[maxima_comb]
            maxima_weights = all_maxima_weights[maxima_comb]

            # At least <min_pivot_points_inbetween> minima need to be between the first and the last maxima
            if np.sum((np.min(maxima_x) <= minima_x) & (minima_x <= np.max(maxima_x))) < min_pivot_points_inbetween:
                continue
            # At least <min_pivot_points_inbetween> maxima need to be between the first and the last minima
            if np.sum((np.min(minima_x) <= maxima_x) & (maxima_x <= np.max(minima_x))) < min_pivot_points_inbetween:
                continue

            # Do linear regression for these maxima
            max_slope, max_intercept, max_r, max_p, max_err = linregress(maxima_x, maxima_y)
            # The coefficient of determination needs to be high enough
            if max_r*max_r <= min_r_squared or max_p >= max_p_value:
                continue

            # Calculate the regression line
            max_line_x = np.arange(np.min(maxima_x), c.shape[0])
            max_line_y = max_intercept + max_slope * max_line_x
            # Calculate the crossing price percentage
            max_crossing_price_perc = np.mean(c[max_line_x] > max_line_y)
            # This needs to be sufficiently small
            if max_crossing_price_perc >= max_crossing_perc:
                continue

            # The latest pivot point should not be too old
            if c.shape[0] - max(np.max(minima_x), np.max(maxima_x)) > max_age_latest_pivot_point:
                continue

            # Calculate a score that makes channel A better than channel B (difficult and can be definitely changed)
            channel_score = (np.sum(minima_weights) * np.sum(maxima_weights)) * min_r*min_r * max_r*max_r * (1-min_p)*(1-max_p)

            if best_channel_score < channel_score:
                best_channel_score = channel_score
                best_lower_trend_line = {
                    "line_x": min_line_x,
                    "line_y": min_line_y,
                    "points_x": minima_x,
                    "points_y": minima_y,
                    "points_weights": minima_weights
                }
                best_lower_trend_stats = {
                    "slope": min_slope,
                    "intercept": min_intercept,
                    "r": min_r,
                    "p": min_p,
                    "err": min_err,
                    "crossing_price_perc": min_crossing_price_perc
                }
                best_upper_trend_line = {
                    "line_x": max_line_x,
                    "line_y": max_line_y,
                    "points_x": maxima_x,
                    "points_y": maxima_y,
                    "points_weights": maxima_weights
                }
                best_upper_trend_stats = {
                    "slope": max_slope,
                    "intercept": max_intercept,
                    "r": max_r,
                    "p": max_p,
                    "err": max_err,
                    "crossing_price_perc": max_crossing_price_perc
                }

    return best_lower_trend_line, best_lower_trend_stats, best_upper_trend_line, best_upper_trend_stats, best_channel_score

In [ ]:
ohlcv = yf.download("TSLA", period="max")

[*********************100%%**********************]  1 of 1 completed


In [ ]:
ohlcv

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-06-29,1.266667,1.666667,1.169333,1.592667,1.592667,281494500
2010-06-30,1.719333,2.028000,1.553333,1.588667,1.588667,257806500
2010-07-01,1.666667,1.728000,1.351333,1.464000,1.464000,123282000
2010-07-02,1.533333,1.540000,1.247333,1.280000,1.280000,77097000
2010-07-06,1.333333,1.333333,1.055333,1.074000,1.074000,103003500
...,...,...,...,...,...,...
2023-10-16,250.050003,255.399994,248.479996,253.919998,253.919998,88917200
2023-10-17,250.100006,257.179993,247.080002,254.850006,254.850006,93562900
2023-10-18,252.699997,254.630005,242.080002,242.679993,242.679993,125147800


In [ ]:
WINDOW_SIZE = 80
PLOT_FUTURE_PERIODS = 40

last_plotted_i = -999

for i in tqdm(range(WINDOW_SIZE, ohlcv.shape[0])):
    o = ohlcv.iloc[i-WINDOW_SIZE+1:i+1, 0].values
    h = ohlcv.iloc[i-WINDOW_SIZE+1:i+1, 1].values
    l = ohlcv.iloc[i-WINDOW_SIZE+1:i+1, 2].values
    c = ohlcv.iloc[i-WINDOW_SIZE+1:i+1, 3].values
    v = ohlcv.iloc[i-WINDOW_SIZE+1:i+1, 5].values

    best_lower_trend_line, best_lower_trend_stats, best_upper_trend_line, best_upper_trend_stats, best_channel_score = get_best_channel(
        h=h, l=l, c=c,
        max_rank_possible=20,
        min_rank_for_detection=3,
        min_pivot_point_count=3,
        max_age_latest_pivot_point=20,
        min_r_squared=0.9,
        max_p_value=0.5,
        max_crossing_perc=0.05,
        min_pivot_points_inbetween=1
    )

    if best_channel_score < 1000:
        continue

    # This is just to prevent plotting the same channels a lot times in a row,
    # so the very first appearance of the channel is shown
    if i - last_plotted_i < 10:
        continue

    print(" best_lower_trend_line: ", best_lower_trend_line)
    print("best_lower_trend_stats: ", best_lower_trend_stats)
    print(" best_upper_trend_line: ", best_upper_trend_line)
    print("best_upper_trend_stats: ", best_upper_trend_stats)
    print("    best_channel_score: ", best_channel_score)

    plot_future_periods = min(i+PLOT_FUTURE_PERIODS+1, ohlcv.shape[0])
    o1 = ohlcv.iloc[i-WINDOW_SIZE+1:plot_future_periods, 0].values
    h1 = ohlcv.iloc[i-WINDOW_SIZE+1:plot_future_periods, 1].values
    l1 = ohlcv.iloc[i-WINDOW_SIZE+1:plot_future_periods, 2].values
    c1 = ohlcv.iloc[i-WINDOW_SIZE+1:plot_future_periods, 3].values
    v1 = ohlcv.iloc[i-WINDOW_SIZE+1:plot_future_periods, 5].values

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                vertical_spacing=0.07, subplot_titles=('OHLC', 'Volume'),
                row_width=[0.15, 0.7], )

    fig.add_trace(go.Candlestick(
        x=np.arange(o1.shape[0]),
        open=o1,
        high=h1,
        low=l1,
        close=c1,
        name="OHLC"),
        row=1, col=1
    )

    fig.add_trace(go.Scatter(x=best_upper_trend_line["points_x"], y=best_upper_trend_line["points_y"], text=['Rank: {}'.format(r) for r in best_upper_trend_line["points_weights"]], mode="markers",opacity=0.7, marker=dict(
            color='Purple',
            size=best_upper_trend_line["points_weights"]/2+10
        ), showlegend=False), row=1, col=1)
    fig.add_trace(go.Scatter(x=best_lower_trend_line["points_x"], y=best_lower_trend_line["points_y"], text=['Rank: {}'.format(r) for r in best_lower_trend_line["points_weights"]], mode="markers", opacity=0.7, marker=dict(
            color='Blue',
            size=best_lower_trend_line["points_weights"]/2+10
        ), showlegend=False), row=1, col=1)
    fig.add_trace(go.Scatter(x=best_upper_trend_line["line_x"], y=best_upper_trend_line["line_y"], opacity=0.7, showlegend=False, text=f"Error: {best_upper_trend_stats['err']}<br>R: {best_upper_trend_stats['r']}<br>P: {best_upper_trend_stats['p']}"), row=1, col=1)
    fig.add_trace(go.Scatter(x=best_lower_trend_line["line_x"], y=best_lower_trend_line["line_y"], opacity=0.7, showlegend=False, text=f"Error: {best_lower_trend_stats['err']}<br>R: {best_lower_trend_stats['r']}<br>P: {best_lower_trend_stats['p']}"), row=1, col=1)

    fig.add_vline(x=WINDOW_SIZE-1, line_width=1, line_dash="dash", line_color="black", row=1, col=1, annotation_text="now")

    fig.add_trace(go.Bar(x=np.arange(o1.shape[0]), y=v1, showlegend=False), row=2, col=1)
    fig.update_layout(height=500, width=800)
    fig.update(layout_xaxis_rangeslider_visible=False)
    fig.show()

    last_plotted_i = i

    if input("Hit any key for the next plot! 'q' for exit.").lower().strip() == "q":
        break

  1%|▏         | 42/3272 [00:01<01:12, 44.38it/s] 

 best_lower_trend_line:  {'line_x': array([44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
       61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       78, 79]), 'line_y': array([1.82794776, 1.81397198, 1.79999621, 1.78602044, 1.77204467,
       1.75806889, 1.74409312, 1.73011735, 1.71614157, 1.7021658 ,
       1.68819003, 1.67421426, 1.66023848, 1.64626271, 1.63228694,
       1.61831116, 1.60433539, 1.59035962, 1.57638384, 1.56240807,
       1.5484323 , 1.53445653, 1.52048075, 1.50650498, 1.49252921,
       1.47855343, 1.46457766, 1.45060189, 1.43662612, 1.42265034,
       1.40867457, 1.3946988 , 1.38072302, 1.36674725, 1.35277148,
       1.3387957 ]), 'points_x': array([44, 53, 69]), 'points_y': array([1.850667  , 1.66666698, 1.49133301]), 'points_weights': array([ 7, 13, 10])}
best_lower_trend_stats:  {'slope': -0.013975772936973652, 'intercept': 2.442881766575042, 'r': -0.9848664537527273, 'p': 0.1108956988678158, 'err': 0.002459430141578320

Hit any key for the next plot! 'q' for exit.


  2%|▏         | 73/3272 [00:09<07:59,  6.67it/s]

 best_lower_trend_line:  {'line_x': array([24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
       41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57,
       58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74,
       75, 76, 77, 78, 79]), 'line_y': array([1.94473855, 1.92936123, 1.91398392, 1.89860661, 1.8832293 ,
       1.86785199, 1.85247468, 1.83709737, 1.82172006, 1.80634274,
       1.79096543, 1.77558812, 1.76021081, 1.7448335 , 1.72945619,
       1.71407888, 1.69870157, 1.68332425, 1.66794694, 1.65256963,
       1.63719232, 1.62181501, 1.6064377 , 1.59106039, 1.57568308,
       1.56030576, 1.54492845, 1.52955114, 1.51417383, 1.49879652,
       1.48341921, 1.4680419 , 1.45266459, 1.43728727, 1.42190996,
       1.40653265, 1.39115534, 1.37577803, 1.36040072, 1.34502341,
       1.3296461 , 1.31426878, 1.29889147, 1.28351416, 1.26813685,
       1.25275954, 1.23738223, 1.22200492, 1.20662761, 1.19125029,
       1.17587298, 1.16049567,

...

Hit any key for the next plot! 'q' for exit.


 66%|██████▌   | 2148/3272 [01:02<01:58,  9.47it/s]

 best_lower_trend_line:  {'line_x': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
       61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       78, 79]), 'line_y': array([19.64627608, 19.56905433, 19.49183258, 19.41461083, 19.33738908,
       19.26016732, 19.18294557, 19.10572382, 19.02850207, 18.95128032,
       18.87405857, 18.79683681, 18.71961506, 18.64239331, 18.56517156,
       18.48794981, 18.41072806, 18.3335063 , 18.25628455, 18.1790628 ,
       18.10184105, 18.0246193 , 17.94739754, 17.87017579, 17.79295404,
       17.71573229, 17.63851054, 17.56128879, 17.48406703, 17.40684528,
       17.32962353, 17.25240178, 17.17518003, 17.09795828, 17.02073652,
       16.94351477, 16.86629302, 16.78907127, 16.71184952, 16.63462777,
       16.55740601, 16.48018426, 16.40296251, 16.32574076, 16

 66%|██████▌   | 2157/3272 [01:08<00:35, 31.42it/s]


KeyboardInterrupt: ignored